<a href="https://colab.research.google.com/github/Valentin-Laurent/MAPIE-DataCraft/blob/main/notebooks/regression-use-case-correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Concrete Use Case of Regression with MAPIE

## Context

California is divided into districts, each requiring a license to operate as a real estate professional.* Every 10 years, a population census is conducted, and the public data is a goldmine for real estate developers in the state of California.

This data contains the following information:

| Variable   | Description |
|------------|------------|
| `MedHouseVal` | Median house value in a given district (in hundreds of thousands of dollars). |
| `MedInc`     | Median household income in the district (in tens of thousands of dollars). |
| `HouseAge`   | Median house age in the district (in years). |
| `AveRooms`   | Average number of rooms per dwelling. |
| `AveBedrms`  | Average number of bedrooms per dwelling. |
| `Population` | Total population of the district. |
| `AveOccup`   | Average number of people per dwelling. |
| `Latitude`   | Latitude of the district center. |
| `Longitude`  | Longitude of the district center. |

Bill Smith, a real-estate developer, seeks your help. Indeed, as with previous censuses, some districts, known as "sneaky" districts, do not wish to publish data related to house values in their area. This information is crucial to estimate whether it is worthwhile to pursue a license in a given district.

The datasets available to you are:
  - `X` and `y`: data from districts that have published all their figures (`y` contains the `MedHouseVal` information, and `X` the other variables)
  - `X_sneaky`: data from sneaky districts (without the `MedHouseVal` information)

During the previous census, Mr. Smith had to mobilize his teams for over a week to estimate this missing data, district by district. This time, he wants to automate the process.

After several workshops with him, you defined a deliverable in the form of a map, highlighting in green the sneaky districts whose median house value does not exceed $150,000. Bill seems rather risk-averse and prefers to obtain a reduced number of districts that we are sure about.

*Note: this use case is fictional, so we have taken liberties with the actual laws in California!

## Action Plan
1. Quick exploratory data analysis
2. Training a GradientBoostingRegressor model, as it is powerful and supports quantile loss (we will see the benefit of this loss later)
3. Conformalizing the model using the ConformalizedQuantileRegressor method
4. Predicting confidence intervals on the X_sneaky dataset for the `MedHouseVal` variable
5. Displaying on a map the sneaky districts that meet Bill's criteria

# Imports and Loading the Dataset

In [ ]:
!rm -rf /content/MAPIE-DataCraft
!git clone https://github.com/Valentin-Laurent/MAPIE-DataCraft

In [ ]:
!pip install mapie

In [ ]:
import json
import sys

sys.path.append('/content/MAPIE-DataCraft/notebooks/use_case_files')

import folium
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform, randint
from mapie.metrics.regression import regression_coverage_score, regression_mean_width_score
from mapie.regression import ConformalizedQuantileRegressor
from mapie.utils import train_conformalize_test_split

In [ ]:
RANDOM_STATE = 42

In [ ]:
X = pd.read_csv("/content/MAPIE-DataCraft/notebooks/use_case_files/X.csv")
y = pd.read_csv("/content/MAPIE-DataCraft/notebooks/use_case_files/y.csv").squeeze("columns")
X_sneaky = pd.read_csv("/content/MAPIE-DataCraft/notebooks/use_case_files/X_sneaky.csv")

# Quick Exploratory Data Analysis (Provided)

## Null Values and Duplicates

Let's check if the X dataset has any null values or duplicates.

In [ ]:
print(X.info(), "\n")
print("Duplicates:", X.duplicated().sum())

## Variable Distribution

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Histogram(x=y)
)
fig.update_layout(
    height=350,
    width=600,
    xaxis_title="Median Price (100,000$)",
    title_text="Target",
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

In [ ]:
rows, cols = 3, 3
fig = make_subplots(rows=rows, cols=cols, subplot_titles=X.columns)

for i, col in enumerate(X.columns):
    row = i // cols + 1
    col_num = i % cols + 1
    fig.add_trace(go.Histogram(y=X[col], name=col), row=row, col=col_num)

fig.update_layout(
    height=700,
    width=1200,
    title_text="Histograms",
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)
fig.show()

## Correlations Between Variables

In [ ]:
corr_matrix = X.corr()

fig = ff.create_annotated_heatmap(
    z=np.abs(corr_matrix.values),
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=corr_matrix.round(2).values,
    showscale=True
)
fig.update_layout(
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

# Training a Predictive Model

## Splitting Training and Conformity Data

For "split" methods included in MAPIE (`SplitConformalRegressor` and `ConformalizedQuantileRegressor`), the training and conformity datasets are used as follows:

![title](https://github.com/Valentin-Laurent/MAPIE-DataCraft/blob/main/notebooks/use_case_files/data-split.png?raw=1)


**Exercise 1**: In the previous tutorial, we used the `ConformalizedQuantileRegressor` by letting MAPIE handle the model training. This time, we will use the `prefit=True` mode with a `GradientBoostingRegressor` trained by us. We need a training set, a test set, and, to use MAPIE, a conformity set.

Create a training set (`X_train`, `y_train`), a conformity set (`X_conformalize`, `y_conformalize`) and a test set (`X_test`, `y_test`) using `train_conformalize_test_split`, and so that:
  - 80% of the data is used for the training;
  - The conformity set and the test set are of equal sizes.

In [ ]:
X_train, X_conformalize, X_test, y_train, y_conformalize, y_test = train_conformalize_test_split(X, y, train_size=0.8, conformalize_size=0.1, test_size=0.1, random_state=RANDOM_STATE)  # correction

## Training

We will use a `GradientBoostingRegressor`, which is quite robust to correlated variables and asymmetric distributions.

**Exercise 2**:
  - Perform a hyperparameter search on a sklearn `GradientBoostingRegressor` model using the `RandomizedSearchCV` method (which allows cross-validation).
  - Do not exceed 50 trainings (this number depends on the parameter grid searched and the number of folds).
  - Use `loss="absolute_error"`

In [ ]:
param_distributions = {
    "n_estimators": randint(50, 500),
    "learning_rate": uniform(0.01, 0.3),
    "max_depth": randint(2, 10),
    "subsample": uniform(0.5, 0.5),
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 20),
    "max_features": uniform(0.5, 0.5)
}

In [ ]:
random_search = RandomizedSearchCV(  # correction
  GradientBoostingRegressor(loss="absolute_error", random_state=RANDOM_STATE),  # correction
  param_distributions,  # correction
  n_iter=15,  # correction
  cv=3,  # correction
  scoring="neg_mean_absolute_error",  # correction
  n_jobs=-1,  # correction
  verbose=1,  # correction
  random_state=RANDOM_STATE,  # correction
)  # correction
random_search.fit(X_train, y_train)  # correction

**Exercise 3**: For the most performant model:
- Display its hyperparameters
- Its average MAE on the cross-validation sets
- Its MAE on the test set
- Interpretation of these values

In [ ]:
regressor = random_search.best_estimator_  # correction
print("Best parameters:", random_search.best_params_, "\n")  # correction
print("MAE on training set", -random_search.best_score_)  # correction
print("On the training set, our model is wrong on average by", round(-random_search.best_score_ * 100), "k$.")  # correction
y_pred = regressor.predict(X_test)  # correction
print("On the test set, our model is wrong on average by", round(mean_absolute_error(y_test, y_pred)*100), "k$.")  # correction
# Our model does not seem to have overfit as the MAE on the training and test sets are similar  # correction

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred,
    mode="markers",
    name="Predictions",
    marker=dict(color="blue", opacity=0.6)
))

min_val, max_val = np.min(y_test), np.max(y_test)
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode="lines",
    name="Line y = x",
    line=dict(color="red", dash="dash")
))

fig.update_layout(
    title="Comparison between y_test and y_pred",
    xaxis_title="y_test (100,000$)",
    yaxis_title="y_pred (100,000$)",
    showlegend=True,
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

fig.show()

## Calculating Intervals with MAPIE

### Explanation of the `ConformalizedQuantileRegressor`
This method requires not 1 but 3 models: a usual model trained to predict the target variable (our model trained above), and two models trained to predict the 5% and 95% quantiles of the target variable, respectively. The idea is that the target variable falls 90% of the time between the predictions of our two quantile models at 5% and 95%.

Finally, the conformalization phase allows "calibrating" these models to provide the theoretical guarantees specific to conformal predictions.

![title](https://github.com/Valentin-Laurent/MAPIE-DataCraft/blob/main/notebooks/use_case_files/quantiles.png?raw=1)

We must therefore train two additional models corresponding to the upper and lower quantiles before using MAPIE.

### Using the `ConformalizedQuantileRegressor`

**Exercise 4**: Train the two quantile models with the same hyperparameters as the main model

In [ ]:
confidence_level = 0.90

In [ ]:
regressor_lower_quantile = GradientBoostingRegressor(  # correction
  **random_search.best_params_,   # correction
  loss="quantile",  # correction
  alpha=(1-confidence_level)/2,  # correction
  random_state=RANDOM_STATE  # correction
).fit(X_train, y_train)  # correction

In [ ]:
regressor_upper_quantile = GradientBoostingRegressor(  # correction
  **random_search.best_params_,  # correction
  loss="quantile",  # correction
  alpha=(1+confidence_level)/2,  # correction
  random_state=RANDOM_STATE  # correction
).fit(X_train, y_train)  # correction

**Exercise 5**: Out of curiosity, let's look at the empirical coverage rates on the test set that these models provide *before* conformalization with MAPIE. Create `y_pred_lower_quantile_before_mapie` and `y_pred_upper_quantile_before_mapie`.

In [ ]:
y_pred_lower_quantile_before_mapie = regressor_lower_quantile.predict(X_test)  # correction
y_pred_upper_quantile_before_mapie = regressor_upper_quantile.predict(X_test)  # correction

In [ ]:
y_pred_combined = np.vstack((y_pred_lower_quantile_before_mapie, y_pred_upper_quantile_before_mapie)).T
print(f"Empirical coverage rate before MAPIE: {regression_coverage_score(y_test, y_pred_combined)[0]:.3f}")
y_pred_combined_expanded = y_pred_combined.reshape(y_pred_combined.shape[0], y_pred_combined.shape[1], 1)
print(f"Average interval width before MAPIE: {round(regression_mean_width_score(y_pred_combined_expanded)[0], 2)} (100k$)")

In [ ]:
df = pd.DataFrame((y_pred_upper_quantile_before_mapie - y_pred_lower_quantile_before_mapie), columns=["Interval Width"])
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=df["Interval Width"]
    )
)
fig.update_layout(
    title_text="Distribution of Interval Widths (100,000$)",
    xaxis_title="Interval Size (100,000$)",
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)
fig.show()

What to think of this empirical coverage rate? Let's now use MAPIE to obtain more robust coverage guarantees:

**Exercise 6**: Use the `ConformalizedQuantileRegressor` in `prefit` mode to predict `y_preds_test` and `y_pred_intervals_test` on `X_test`

In [ ]:
mapie_regressor = ConformalizedQuantileRegressor(  # correction
  estimator=[regressor_lower_quantile, regressor_upper_quantile, regressor],  # correction
  confidence_level=confidence_level,  # correction
  prefit=True  # correction
)  # correction
mapie_regressor.conformalize(X_conformalize, y_conformalize)  # correction

In [ ]:
y_preds_test, y_pred_intervals_test = mapie_regressor.predict_interval(  # correction
  X_test  # correction
)  # correction

Let's check our empirical coverage rate on the test set and calculate the average size of our intervals:

In [ ]:
y_pred_lower_quantile = y_pred_intervals_test[:, 0, 0]
y_pred_upper_quantile = y_pred_intervals_test[:, 1, 0]

y_pred_combined = np.vstack((y_pred_lower_quantile, y_pred_upper_quantile)).T
print(f"Empirical coverage rate: {regression_coverage_score(y_test, y_pred_combined)[0]:.3f}")
y_pred_combined_expanded = y_pred_combined.reshape(y_pred_combined.shape[0], y_pred_combined.shape[1], 1)
print(f"Average interval width: {round(regression_mean_width_score(y_pred_combined_expanded)[0], 2)} (100 000$)")

intervals_before = (y_pred_upper_quantile_before_mapie - y_pred_lower_quantile_before_mapie)
intervals_after = (y_pred_upper_quantile - y_pred_lower_quantile)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.15,
                    subplot_titles=["Before MAPIE", "After MAPIE"])
fig.add_trace(go.Histogram(x=intervals_before, name="Before MAPIE", nbinsx=20), row=1, col=1)
fig.add_trace(go.Histogram(x=intervals_after, name="After MAPIE", nbinsx=20), row=2, col=1)
fig.update_layout(
    title_text="Distribution of Interval Widths (100,000$)",
    showlegend=False,
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)
fig.show()

Is the coverage rate satisfactory? Is it at the expense of another indicator of interest in our problem?

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test[0::30],
    y=y_pred[0::30],
    error_y=dict(
        type="data",
        array=intervals_after,
    ),
    mode="markers",
    name="Predictions",
    marker=dict(color="blue", opacity=0.6)
))

min_val, max_val = np.min(y_test), np.max(y_test)
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode="lines",
    name="Line y = x",
    line=dict(color="red", dash="dash")
))

fig.update_layout(
    title="Comparison between y_test and y_pred with error bars (on a subset for better readability)",
    xaxis_title="y_test (100,000$)",
    yaxis_title="y_pred (100,000$)",
    showlegend=True,
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")
)

fig.show()

## Analysis of Green Districts on the Test Set

Remember: the agreed deliverable with Bill is a map highlighting in green the sneaky districts whose median house value does not exceed $150,000. And Bill prefers to obtain a reduced number of districts that we are sure about.

We can, for example, define "green districts" as follows: those whose entire interval predicted by MAPIE is below the $150k threshold:

![title](https://github.com/Valentin-Laurent/MAPIE-DataCraft/blob/main/notebooks/use_case_files/green-districts.png?raw=1)

**Exercise 7** - difficult ;)
- We chose a coverage level of 90%. What statistical guarantee can we communicate to Bill about the green districts?
- We can, for example, look at the coverage rate on green districts only. Create a Numpy boolean array named `green_districts_mask` from `y_pred_upper_quantile`. An element of this array is `True` if the district meets the condition given above, and `False` otherwise. Then, calculate the coverage rate on green districts only.

In [ ]:
# The coverage rate applies to all our intervals. That is, on average across ALL districts, our coverage rate will be statistically above 90%.  # correction
# The coverage rate on green districts can therefore be different. We talk about "conditional coverage". We can estimate an empirical rate by looking at its value on the test set:  # correction
green_districts_mask = y_pred_upper_quantile <= 1.5  # correction
y_pred_upper_quantile_green = y_pred_upper_quantile[green_districts_mask]  # correction
y_pred_lower_quantile_green = y_pred_lower_quantile[green_districts_mask]  # correction
y_pred_upper_quantile_red = y_pred_upper_quantile[~green_districts_mask]  # correction
y_pred_lower_quantile_red = y_pred_lower_quantile[~green_districts_mask]  # correction
y_test_green = y_test[green_districts_mask]  # correction
y_test_red = y_test[~green_districts_mask]  # correction
y_pred_green = np.vstack((y_pred_lower_quantile_green, y_pred_upper_quantile_green)).T   # correction
y_pred_red = np.vstack((y_pred_lower_quantile_red, y_pred_upper_quantile_red)).T  # correction
fig = go.Figure(go.Bar(  # correction
    x=["Coverage Rate of Non-Green Districts", "Coverage Rate of Green Districts"],  # correction
    y=[  # correction
        regression_coverage_score(y_test_red, y_pred_red)[0],  # correction
        regression_coverage_score(y_test_green, y_pred_green)[0]  # correction
    ],  # correction
    text=[f"{len(y_test_red)} districts", f"{len(y_test_green)} districts"],  # correction
    textposition="auto",  # correction
    marker=dict(color=["red", "green"])  # correction
))  # correction
fig.update_layout(  # correction
    title="Empirical Coverage Rates on the Test Set",  # correction
    #xaxis_title="Series",  # correction
    #yaxis_title="Number of observations",  # correction
    font=dict(family="Computer Modern", size=18, color="#7f7f7f")  # correction
)  # correction
fig.show()  # correction

Our coverage rate seems better when it comes to green districts! This is reassuring information, even though the only statistical guarantee we can provide to Bill is on **all** sneaky districts. #  correction

# Producing the Deliverable in the Form of a Map

**Exercise 8**:
- Use MAPIE to calculate `y_preds_sneaky` and `y_pred_intervals_sneaky`
- Create a Numpy array `green_sneaky_districts` containing the list of sneaky districts we want to display (`True`) or not (`False`) from `y_pred_intervals_sneaky`
- Create a DataFrame `map_data` containing the columns `Latitude`, `Longitude`, and `Green district`
- Use the folium code to display the map requested by Bill

In [ ]:
y_preds_sneaky, y_pred_intervals_sneaky = mapie_regressor.predict_interval(X_sneaky)  # correction
green_sneaky_districts = y_pred_intervals_sneaky[:, 1, 0] <= 1.5  # correction
map_data = X_sneaky[["Latitude", "Longitude"]].copy()  # correction
map_data["Green district"] = green_sneaky_districts  # correction

In [ ]:
california_map = folium.Map(location=[36.7783, -119.4179], zoom_start=6, min_zoom=5)

for _, row in map_data.iterrows():
    if row["Green district"]:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3,
            color="green",
            fill=True,
            fill_opacity=0.7,
        ).add_to(california_map)

n_sneaky_district_total = X_sneaky.shape[0]
n_green_district = map_data["Green district"].sum()
print("Total number of sneaky districts", n_sneaky_district_total)
print("Number of green districts", n_green_district)
print("Number of districts not displayed on the map", n_sneaky_district_total - n_green_district)

california_map

# Feedback from Bill

Bill is satisfied with this first deliverable and understands the statistical significance of your results. He wants to continue working with you to refine the work provided.

In particular, he would like to know the sneaky districts that are uncertain but promising. That is, those where:
 - the house value predicted by the model is below $150,000
 - but the upper bound predicted by MAPIE exceeds this threshold

You have educated Bill on the limitations of AI, and he understands that he will have to make a case-by-case decision with his teams for these districts.

Let's refine the deliverable by displaying these uncertain districts in orange.

![title](https://github.com/Valentin-Laurent/MAPIE-DataCraft/blob/main/notebooks/use_case_files/orange-districts.png?raw=1)

**Exercise 9**:
- Create a Numpy array `orange_sneaky_districts` containing the list of uncertain districts we want to display (`True`) or not (`False`) from `y_preds_sneaky` and `y_pred_intervals_sneaky`.
- Add the `Orange district` column to the `map_data` DataFrame
- Display the new map

In [ ]:
orange_sneaky_districts = (y_preds_sneaky <= 1.5) & (y_pred_intervals_sneaky[:, 0, 0] <= 1.5) & (y_pred_intervals_sneaky[:, 1, 0] > 1.5)  # correction
map_data["Orange district"] = orange_sneaky_districts  # correction

In [ ]:
california_map = folium.Map(location=[36.7783, -119.4179], zoom_start=6, min_zoom=5)

for _, row in map_data.iterrows():
    if row["Green district"]:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3,
            color="green",
            fill=True,
            fill_opacity=0.7,
        ).add_to(california_map)

for _, row in map_data.iterrows():
    if row["Orange district"]:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=1,
            color="orange",
            fill=True,
            fill_opacity=0.7,
        ).add_to(california_map)

n_orange_district = map_data["Orange district"].sum()
print("Total number of sneaky districts", n_sneaky_district_total)
print("Number of green districts", n_green_district)
print("Number of orange districts", n_orange_district)
print("Number of districts not displayed on the map", n_sneaky_district_total - n_green_district - n_orange_district)

california_map

Congratulations! You are now an expert in conformal predictions ;)

# To Go Further

Some open questions to further reflect on this use case:
1. Imagine Bill is not risk-averse but rather a daredevil: What confidence level choices would you make? What choice between green and orange districts?
2. Bill informs us that the total number of rooms is an indicator of interest in his profitability calculations. We can look at the conditional coverage rate based on the total number of rooms (for example, by dividing this variable into categories: 0-10, 10-20, 30-40, 40-50). Are the coverage rates corresponding to these different categories homogeneous? If not, what remediation method could we use?